In [1]:
import config
from bs4 import BeautifulSoup as BS
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
import time
import re
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('updated_df.csv', index_col=0)

In [3]:
df.head()

,fit,user_id,bust_size,item_id,weight,rating,rented_for,review_text,body_type,review_summary,category,height,size,age,cup_size
0,fit,420272,34,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,d
1,fit,273551,34,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,b
3,fit,909926,34,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,c
4,fit,151944,34,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,b
5,fit,734848,32,364092,138lbs,8.0,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68.0,8,45.0,b


In [4]:
#### change weight column to drop 'lbs'
df['weight'] = df['weight'].str.split('\D+')
df['weight'] = df['weight'].apply(lambda x:x[0])
df['weight'] = df['weight'].astype('int32')

In [5]:
df.head()

,fit,user_id,bust_size,item_id,weight,rating,rented_for,review_text,body_type,review_summary,category,height,size,age,cup_size
0,fit,420272,34,2260466,137,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,d
1,fit,273551,34,153475,132,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,b
3,fit,909926,34,126335,135,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,c
4,fit,151944,34,616682,145,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,b
5,fit,734848,32,364092,138,8.0,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68.0,8,45.0,b


In [6]:
### remove outlier from "rented_for" columns
condition2 = df['rented_for'] != 'party: cocktail'
df = df[condition2]

In [ ]:
df.head()

In [7]:
df.category.value_counts()

dress           70341
gown            33240
sheath          14679
shift            4032
jumpsuit         4016
                ...  
sweatershirt        3
sweatpants          1
crewneck            1
buttondown          1
overcoat            1
Name: category, Length: 68, dtype: int64

In [8]:
df.category.unique()

array(['romper', 'gown', 'dress', 'sheath', 'leggings', 'sweater',
       'jacket', 'shirtdress', 'jumpsuit', 'shift', 'top', 'shirt',
       'mini', 'skirt', 'maxi', 'pants', 'suit', 'coat', 'blouse',
       'trench', 'cape', 'bomber', 'blazer', 'vest', 'duster', 'ballgown',
       'tank', 'poncho', 'frock', 'tunic', 'cardigan', 'down', 'culottes',
       'midi', 'legging', 'print', 'pant', 'knit', 'culotte',
       'sweatshirt', 'peacoat', 'trouser', 'kaftan', 'overalls', 'jogger',
       'tee', 'combo', 'henley', 'blouson', 'pullover', 'turtleneck',
       'trousers', 'overcoat', 'hoodie', 't-shirt', 'caftan', 'tight',
       'kimono', 'cami', 'for', 'crewneck', 'skirts', 'parka',
       'buttondown', 'skort', 'sweatershirt', 'jeans', 'sweatpants'],
      dtype=object)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'bust size': 'bust_size', 'rented for': 'rented_for', 'body type': 'body_type'})

In [ ]:
df.bust_size.value_counts()

In [ ]:
re.split('(\D+)', '34b')

In [ ]:
df['bust_size'] = df['bust_size'].str.split('(\D+)')

In [ ]:
df.head()

In [ ]:
df['cup_size'] = df.bust_size.str[1]

In [ ]:
df['bust_size'] = df.bust_size.str[0]

In [ ]:
#df = df.dropna()

In [ ]:
condition = df['height'] != 'nan'

In [ ]:
df = df[condition]

In [ ]:
def parse_ht(ht):
    # format: 7' 0.0"
    ht_ = ht.split("' ")
    ft_ = float(ht_[0])
    in_ = float(ht_[1].replace("\"",""))
    return (12*ft_) + in_

#print df["HeightNoShoes"].apply(lambda x:parse_ht(x))

In [ ]:
parse_ht("5' 8")

In [ ]:
df['height'] = df['height'].astype(str)

In [ ]:
df['height'] = df['height'].str.split('"')

In [ ]:
df['height'] = df['height'].apply(lambda x:x[0])

In [ ]:
df.height[0]

In [ ]:
df['height'] = df['height'].apply(lambda x:parse_ht(x))

In [ ]:
df.head()

In [ ]:
df.to_csv(r'/Users/andrewtriola/Documents/flatiron/mod_6_proj/mod_6_project/updated_df.csv')

In [9]:
corr_df = df.corr()

In [ ]:
df['rating'].value_counts()

In [ ]:
plt.hist(df['height'], bins=10)

In [ ]:
df['weight']

In [ ]:
abc = re.split('\D+', '137lbs')[0]

In [ ]:
abc

In [ ]:
df['weight'] = df['weight'].str.split('\D+')

In [ ]:
df['weight'] = df['weight'].apply(lambda x:x[0])

In [ ]:
df['weight'] = df['weight'].astype('int32')

In [ ]:
df.head()

In [ ]:
df.to_csv(r'/Users/andrewtriola/Documents/flatiron/mod_6_proj/mod_6_project/updated_df2.csv')

In [ ]:
plt.hist(df['weight'], bins=25)

In [ ]:
df['user_id'].value_counts()

In [ ]:
condition1 = df['user_id'] == 45337

In [ ]:
condition2 = df['rented_for'] != 'party: cocktail'

In [ ]:
df = df[condition2]

In [ ]:
##regroup categories (reduce from 68 to 7)
recat1 = df.replace(['dress', 'sheath', 'shirtdress', 'shift', 'ballgown', 'frock', 'kaftan', 'caftan', 'gown', 'print'], 'dresses')
recat2 = recat1.replace(['romper', 'jumpsuit', 'overalls', 'combo', 'suit'], 'jumpsuits')
recat3 = recat2.replace(['jogger', 'trousers', 'tight', 'jeans', 'sweatpants', 'leggings', 'pants', 'culottes', 'legging', 'pant', 'culotte', 'trouser'], 'pants')
recat4 = recat3.replace(['sweater', 'duster', 'cardigan', 'sweatshirt', 'pullover', 'turtleneck', 'hoodie', 'sweatershirt'], 'sweaters')
recat5 = recat4.replace(['jacket', 'coat', 'trench', 'cape', 'bomber', 'blazer', 'vest', 'poncho', 'down', 'parka', 'peacoat', 'overcoat'], 'outerwear')
recat6 = recat5.replace(['top,', 'shirt', 'blouse', 'tank', 'tunic', 'knit', 'tee', 'henley', 'blouson', 't-shirt', 'kimono', 'cami', 'crewneck', 'buttondown', 'for'], 'tops')
recat7 = recat6.replace(['mini', 'skirt', 'maxi','midi', 'skirts', 'skort'], 'skirts')
recat8 = recat7.replace(['top'], 'tops')

##rename recategorized df variable
df = recat8

In [11]:
dress_condition = ((df['category'] == 'dress') | (df['category'] == 'sheath') | (df['category'] == 'shirtdress') | 
(df['category'] == 'shift') | (df['category'] == 'ballgown') | (df['category'] == 'frock') | 
(df['category'] == 'kaftan') | (df['category'] == 'caftan') | (df['category'] == 'gown') | (df['category'] == 'print'))

In [12]:
dress_df = df[dress_condition]

In [13]:
dress_df.head()

,fit,user_id,bust_size,item_id,weight,rating,rented_for,review_text,body_type,review_summary,category,height,size,age,cup_size
1,fit,273551,34,153475,132,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,b
3,fit,909926,34,126335,135,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,c
4,fit,151944,34,616682,145,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,b
5,fit,734848,32,364092,138,8.0,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68.0,8,45.0,b
6,fit,336066,34,568429,112,10.0,everyday,This dress is so sweet. I loved the print. The...,hourglass,LITERALLY THE CUTEST DRESS EVER,dress,63.0,4,27.0,c


In [ ]:
df.category.value_counts()

In [14]:
user_df = df[['user_id', 'bust_size', 'cup_size', 'body_type', 'weight', 'height', 'age']].copy()

In [15]:
item_df = df[['item_id', 'rating', 'rented_for', 'category']].copy()

In [16]:
dress_item_df = dress_df[['item_id', 'rating', 'rented_for', 'category']].copy()

In [ ]:
item_df.shape

In [ ]:
user_df.head()

In [22]:
ratings_df = small_df[['user_id', 'item_id', 'rating']]

In [23]:
dress_ratings_df = dress_df[['user_id', 'item_id', 'rating']]

In [ ]:
dress_ratings_df.head()

In [ ]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(2.0, 10.0))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(dress_ratings_df, reader)

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
### grid search ###

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
###one hot encode item df
items_dummies = pd.get_dummies(item_df)
items_dummies.head()

In [ ]:
###one hot encode item df
dress_items_dummies = pd.get_dummies(dress_item_df)
dress_items_dummies.head()

In [ ]:
dress_items_dummies.shape

In [ ]:
# cosine_similarity(items_dummies)

In [ ]:
items_df_dress = dress_items_dummies.groupby(['item_id']).mean()

In [ ]:
df.groupby(['item_id']).item_id.count().sort_values(ascending = False).head(50)

In [19]:
df['item_count'] = df.groupby(['item_id'])['item_id'].transform('count')

In [ ]:
df.head()

In [20]:
condition3 = df['item_count'] < 20

In [ ]:
len((df[df['item_count'] < 20])['item_id'].unique())

In [21]:
small_df = df[condition3]

In [ ]:
items_df_dress.shape

In [ ]:
items_df_dress.head()

In [ ]:
cosine_similarity(items_df_dress)

In [ ]:
### make rec matrix for dress df

In [ ]:
dress_recs = dress_df[['user_id', 'item_id', 'rating']]

In [ ]:
reader = Reader(rating_scale=(2,10))
data = Dataset.load_from_df(dress_recs,reader)

In [ ]:
train, test = train_test_split(data,test_size=0.25)

In [ ]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

In [ ]:
### grid search ###

param_grid = {'n_epochs': [12,13], 'lr_all': [0.009, 0.01, 0.011],
              'reg_all': [0.35,0.4, 0.45]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# skipping
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))



In [ ]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
np.mean(cv_knn_baseline['test_rmse'])

In [25]:
dress_df['item_count'] = dress_df.groupby(['item_id'])['item_id'].transform('count')

/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
condition4 = dress_df['item_count'] < 20

In [28]:
small_dress_df = dress_df[condition4]

In [45]:
dress_small_ratings_df = small_dress_df[['user_id', 'item_id', 'rating']]

In [46]:
reader = Reader(rating_scale=(2,10))
data = Dataset.load_from_df(dress_small_ratings_df,reader)

In [32]:
# skipping
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))



('test_rmse', array([1.49949173, 1.51515135, 1.52755969, 1.53797294, 1.537429  ]))
('test_mae', array([1.23950095, 1.25203504, 1.25227433, 1.25828419, 1.25664369]))
('fit_time', (20.733392000198364, 20.568089723587036, 20.11698579788208, 18.573551177978516, 5.321741104125977))
('test_time', (0.17890191078186035, 0.20891213417053223, 0.10373902320861816, 0.11112499237060547, 0.046755075454711914))
-----------------------
1.5235209408682269


In [34]:
# cross validating with KNNBaseline
from surprise.prediction_algorithms import KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
np.mean(cv_knn_baseline['test_rmse'])

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.49987156, 1.45807163, 1.46994754, 1.51157683, 1.48477858]))
('test_mae', array([1.20052122, 1.1659273 , 1.1856336 , 1.20520975, 1.18634309]))
('fit_time', (5.200239896774292, 5.399817943572998, 5.261378765106201, 5.1657209396362305, 5.300236225128174))
('test_time', (0.056257009506225586, 0.055994272232055664, 0.06219911575317383, 0.06447601318359375, 0.18319416046142578))


1.4848492272562706

In [37]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

In [38]:
# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.4974


1.4974160703146928

In [48]:
dress_big_ratings_df = dress_df[['user_id', 'item_id', 'rating']]

In [50]:
dress_big_ratings_df.head()

,user_id,item_id,rating
1,273551,153475,10.0
3,909926,126335,8.0
4,151944,616682,10.0
5,734848,364092,8.0
6,336066,568429,10.0


In [51]:
# index=user columns=item values=rating
dress_recs = pd.pivot_table(dress_big_ratings_df, values='rating', index='user_id', columns='item_id')

In [56]:
similar = dress_recs.corrwith(dress_recs[126335])
corr_similar = pd.DataFrame(similar,columns=['correlation'])

In [57]:
corr_similar.head()

,correlation
item_id,
123373,-0.454545
123793,NaN
124204,0.133631
124553,NaN
125424,0.707107


In [63]:
def get_recs(item, n_recs=5):
    similar = dress_recs.corrwith(dress_recs[item])
    corr_similar = pd.DataFrame(similar,columns=['correlation'])
    corr_similar.dropna(inplace=True)
    
    orig = dress_big_ratings_df.copy()
    
    corr_with_item = pd.merge(
            left=corr_similar,
            right=orig,
            on='item_id')[['item_id', 'correlation']].drop_duplicates().reset_index(drop=True)
    
    result = corr_with_item.sort_values(by='correlation', ascending=False)
    
    return result.head(n_recs)

In [64]:
get_recs(126335, n_recs=20)

,item_id,correlation
44,167112,1.0
17,135459,1.0
5,126335,1.0
101,1209909,1.0
27,144727,1.0
39,156607,1.0
29,146684,1.0
33,149655,1.0
78,651557,1.0
42,164593,1.0
